In [2]:
# ! pip install xgboost

In [1]:
# ! pip install vecstack

In [12]:
# ! pip install lightgbm

In [21]:
from sklearn.datasets import load_iris 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier 
from vecstack import stacking
from lightgbm import LGBMClassifier
import numpy as np

**파이썬으로 구현한 스태킹 앙상블 기본코드**  
sklearn에서 제공해주는 암(cancer)데이터를 사용하겠습니다.

In [5]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

X_data = data.data
y_data = data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state = 0)

스태킹 앙상블에서는 2가지 개념의 머신러닝 모델이 필요하다 했죠?  
저는 개별 모델은 SVM, 랜덤 포레스트, 로지스틱 회귀를 사용했고, 최종모델로 LightGBM을 사용했습니다.

In [14]:
# 개별 모델들
svm = SVC(random_state = 0)
rf = RandomForestClassifier(n_estimators =100, random_state=0)
lr = LogisticRegression()

# 최종 모델
lgbm = LGBMClassifier()

svm.fit(X_train, y_train)
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

c:\users\yuna1\appdata\local\continuum\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
svm_pred = svm.predict(X_test)
rf_pred = rf.predict(X_test)
lr_pred = lr.predict(X_test)

print("svm : {0:.4f}, rf : {1:.4f}, lr : {2:.4f}".format( accuracy_score(y_test, svm_pred),accuracy_score(y_test, rf_pred),accuracy_score(y_test, lr_pred)))

svm : 0.9298, rf : 0.9649, lr : 0.9474


자 이걸 합쳐줘야합니다. 합쳐주는 과정이 데이터 쌓는 듯한 모습이라서 stacking이라고 불립니다.

In [22]:
new_data = np.array([svm_pred, rf_pred, lr_pred])
new_data.shape

(3, 114)

In [23]:
new_data = np.transpose(new_data)
new_data.shape

(114, 3)

stacking할 때 데이터의 shape에 주의해야합니다.  
3개의 모델을 stacking하고, 이 값은 predict로 나온 값이기 때문에 row는 X_test와 일치해야 합니다.  
즉, 여기서 X_test의 row는 114개입니다. 이 114개의 데이터가 다시 새로운 훈련 데이터가 됩니다.

In [24]:
new_data[:10]

array([[1, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

이제 마지막 최종 모델을 사용해서 훈련하면 끝!

In [25]:
lgbm.fit(new_data, y_test)
lgbm_pred = lgbm.predict(new_data)
print('정확도 : {0:.4f}'.format(accuracy_score(y_test, lgbm_pred)))

정확도 : 0.9737
